In [143]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from tqdm import tqdm
import matplotlib.pyplot as plt

In [47]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
data = pd.read_csv(
    "D:\projects_2023\main\datasets\stock_exchange_data\indexProcessed.csv")

In [10]:
data.head()

,Index,Date,Open,High,Low,Close,Adj Close,Volume,CloseUSD
0,HSI,1986-12-31,2568.300049,2568.300049,2568.300049,2568.300049,2568.300049,0.0,333.879006
1,HSI,1987-01-02,2540.100098,2540.100098,2540.100098,2540.100098,2540.100098,0.0,330.213013
2,HSI,1987-01-05,2552.399902,2552.399902,2552.399902,2552.399902,2552.399902,0.0,331.811987
3,HSI,1987-01-06,2583.899902,2583.899902,2583.899902,2583.899902,2583.899902,0.0,335.906987
4,HSI,1987-01-07,2607.100098,2607.100098,2607.100098,2607.100098,2607.100098,0.0,338.923013


In [11]:
data = data[data['Index']=='GDAXI']


In [12]:
data = data.drop(["Index", ], axis=1)
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,CloseUSD
79900,1987-12-30,1005.190002,1005.190002,1005.190002,1005.190002,1005.190002,0.0,1226.331802
79901,1988-01-04,956.489990,956.489990,956.489990,956.489990,956.489990,0.0,1166.917788
79902,1988-01-05,996.099976,996.099976,996.099976,996.099976,996.099976,0.0,1215.241971
79903,1988-01-06,1006.010010,1006.010010,1006.010010,1006.010010,1006.010010,0.0,1227.332212
79904,1988-01-07,1014.469971,1014.469971,1014.469971,1014.469971,1014.469971,0.0,1237.653365


In [13]:
data = data.set_index("Date")

In [14]:
data.head()

,Open,High,Low,Close,Adj Close,Volume,CloseUSD
Date,,,,,,,
1987-12-30,1005.190002,1005.190002,1005.190002,1005.190002,1005.190002,0.0,1226.331802
1988-01-04,956.489990,956.489990,956.489990,956.489990,956.489990,0.0,1166.917788
1988-01-05,996.099976,996.099976,996.099976,996.099976,996.099976,0.0,1215.241971
1988-01-06,1006.010010,1006.010010,1006.010010,1006.010010,1006.010010,0.0,1227.332212
1988-01-07,1014.469971,1014.469971,1014.469971,1014.469971,1014.469971,0.0,1237.653365


In [16]:
features = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
target = 'CloseUSD'

In [17]:
X = data[features].values
Y = data[target].values

In [18]:
n_features = X.shape[1]
lookback = 30

In [21]:
X_organized, Y_organized = [], []
for i in range(0, X.shape[0]-lookback, 1):
    X_organized.append(X[i:i+lookback])
    Y_organized.append(Y[i+lookback])

X_organized, Y_organized = np.array(X_organized), np.array(Y_organized)
X_organized, Y_organized = torch.tensor(X_organized, dtype=torch.float32), torch.tensor(Y_organized, dtype=torch.float32)

X_train, X_test, Y_train, Y_test = train_test_split(X_organized, Y_organized, test_size=0.2, random_state=42)

#X_organized.shape, Y_organized.shape,  X_train.shape, Y_train.shape, X_test.shape, Y_test.shape
#X_train, Y_train, X_test, Y_test = X_organized[:50000], Y_organized[:50000], X_organized[50000:], Y_organized[50000:]

In [22]:
mean, std = Y_train.mean(), Y_train.std()
Y_train_scaled, Y_test_scaled = (Y_train - mean)/std , (Y_test-mean)/std

In [95]:
train_data = TensorDataset(X_train, Y_train_scaled)
test_data = TensorDataset(X_test, Y_test_scaled)

batch_size = 64

train_loader = DataLoader(train_data, shuffle=False, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size)

In [31]:

hidden_size = 256
num_layers = 2

In [108]:
class LSTMRegression(nn.Module):
    def __init__(self):
        super(LSTMRegression, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size = n_features, hidden_size =hidden_size,
                            num_layers = num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, 64)
        self.fc2 = nn.Linear(64, 1)

        #self.activation = nn.LeakyReLU()
        #self.activation = nn.ELU()
        self.bn = nn.BatchNorm1d(64)
        self.activation = nn.ReLU()

    def forward(self, x):
        hidden= torch.randn(num_layers, len(x), hidden_size).to(device)
        carry = torch.randn(num_layers, len(x), hidden_size).to(device)
        out, (hidden, carry) = self.lstm(x, (hidden, carry))
        out = self.activation(self.fc1(out[:,-1]))
        out = self.bn(out)
        out = self.fc2(out)
        return out

In [109]:
model = LSTMRegression()

model

LSTMRegression(
  (lstm): LSTM(6, 256, num_layers=2, batch_first=True)
  (fc1): Linear(in_features=256, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (bn): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation): ReLU()
)

In [83]:
for layer in model.children():
    print("Layer : {}".format(layer))
    print("Parameters : ")
    for param in layer.parameters():
        print(param.shape)
    print()

Layer : LSTM(6, 256, num_layers=2, batch_first=True)
Parameters : 
torch.Size([1024, 6])
torch.Size([1024, 256])
torch.Size([1024])
torch.Size([1024])
torch.Size([1024, 256])
torch.Size([1024, 256])
torch.Size([1024])
torch.Size([1024])

Layer : Linear(in_features=256, out_features=64, bias=True)
Parameters : 
torch.Size([64, 256])
torch.Size([64])

Layer : Linear(in_features=64, out_features=1, bias=True)
Parameters : 
torch.Size([1, 64])
torch.Size([1])

Layer : LeakyReLU(negative_slope=0.01)
Parameters : 



In [84]:
def train_lstm(model, train_loader, test_loader, loss, optimizer, epochs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    #model.to(device)

    for epoch in range(1, epochs+1):
        train_loss = []
        test_loss = []
        model.train()
        for batch_x, batch_y in tqdm(train_loader):
            optimizer.zero_grad()
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)
            output = model(batch_x)
            l = loss(output.ravel(), batch_y)
            train_loss.append(l.item())
            l.backward()
            optimizer.step()

        print("Train Loss : {:.3f}".format(torch.tensor(train_loss).mean()))

        with torch.no_grad():
            model.eval()
            for batch_x, batch_y in test_loader:
                batch_x = batch_x.to(device)
                batch_y = batch_y.to(device)
                output = model(batch_x)
                l = loss(output.ravel(), batch_y)
                test_loss.append(l.item())

        print("Test Loss : {:.3f}".format(torch.tensor(test_loss).mean()))

In [115]:
# Hyperparameters

epochs = 10
lr = 1e-4
#weight_decay = 1e-7

In [116]:
loss = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=float(lr))

model = model.to(device)

In [117]:
train_lstm(model, train_loader, test_loader, loss, optimizer, epochs)

100%|██████████| 106/106 [00:01<00:00, 88.61it/s] 


Train Loss : 0.509
Test Loss : 0.480


100%|██████████| 106/106 [00:00<00:00, 106.94it/s]


Train Loss : 0.509
Test Loss : 0.478


100%|██████████| 106/106 [00:01<00:00, 98.58it/s]


...

Train Loss : 0.508
Test Loss : 0.478


100%|██████████| 106/106 [00:01<00:00, 103.39it/s]


Train Loss : 0.508
Test Loss : 0.477


100%|██████████| 106/106 [00:01<00:00, 97.58it/s]


Train Loss : 0.507
Test Loss : 0.478


In [151]:
X_test = X_test.to(device)
test_preds = model(X_test)
test_preds  = (test_preds*std) + mean
test_preds = test_preds.cpu().detach().numpy().squeeze()

print(test_preds.shape)

(1682,)


In [154]:
print("Test  MSE : {:.2f}".format(mean_squared_error(test_preds, Y_test.detach().numpy())))
print("Test  R^2 Score : {:.2f}".format(r2_score(test_preds, Y_test.detach().numpy()))) # Close to 1 is good model

Test  MSE : 9431338.00
Test  R^2 Score : 0.06


In [155]:
data_final = X_test.cpu().detach().numpy().reshape(-1, n_features)
df = pd.DataFrame(data_final, columns=[f'feature_{i}' for i in range(n_features)])
df_close = pd.DataFrame(data_final[:, 0], columns=['CloseUSD'])
df_close['CloseUSD Prediction'] = test_preds
df_close.set_index(df.index, inplace=True)

df_close.plot(y=["CloseUSD", "CloseUSD Prediction"], figsize=(18,7))
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')


ValueError: Length of values (1682) does not match length of index (50460)